# WISDEM Tutorial 1

As an example of WISDEM, let us use the NREL Cost and Scaling Model and simulate the NREL 5MW Reference Model [FAST2009] in a 500 mW offshore plant.  This tutorial is based on the upcoming release of WISDEM.  For tutorials on WISDEM in the current release version (using OpenMDAO 0.10.3.2), see the WISDEM documentation which can be found here:
http://wisdem.github.io/WISDEM/

First navigate to your folder with your WISDEM code.

In [15]:
cd C:\SystemsEng\WISDEM Tutorial\WISDEM\Code Files 2


C:\SystemsEng\WISDEM Tutorial\WISDEM\Code Files 2


The first step is to import the relevant files and set up the component.

In [16]:
# A simple test of WISDEM using the NREL CSM
from openmdao.api import IndepVarComp, Component, Problem, Group
from fused_wind import create_interface , FUSED_Object , FUSED_OpenMDAO , set_output, set_input, fusedvar

# NREL cost and scaling model sub-assemblies
from nrel_csm_tcc import tcc_csm_fused
from nrel_csm_bos import bos_csm_fused
from nrel_csm_opex  import opex_csm_fused
from nrel_csm_fin import fin_csm_fused
from nrel_csm_aep import aep_csm_fused

import numpy as np

Now we will set up a problem that uses the above NREL cost and scaling model (csm) components into a group and add it to a problem for analysis.

In [17]:
# openmdao example of execution
root = Group()
root.add('desvars',IndepVarComp([('machine_rating',5000.0),
                                 ('rotor_diameter', 126.0),
                                 ('hub_height', 90.0),
                                 ('turbine_number', 100.0),
                                 ('year', 2009.0),
                                 ('month',12.0),
                                 ]),promotes=['*'])
root.add('bos_csm_test', FUSED_OpenMDAO(bos_csm_fused()), promotes=['*'])
root.add('tcc_csm_test', FUSED_OpenMDAO(tcc_csm_fused()), promotes=['*'])
root.add('fin_csm_test', FUSED_OpenMDAO(fin_csm_fused()), promotes=['*'])
root.add('bos_opex_test', FUSED_OpenMDAO(opex_csm_fused()), promotes=['*'])
root.add('aep_test', FUSED_OpenMDAO(aep_csm_fused()), promotes=['*'])
prob = Problem(root)
prob.setup()

##############################################
Setup: Checking root problem for potential issues...

No recorders have been specified, so no data will be saved.

The following parameters have no associated unknowns:
RNA_mass
air_density
altitude
array_losses
availability
blade_number
cut_in_wind_speed
cut_out_wind_speed
max_efficiency
max_power_coefficient
max_tip_speed
multiplier
opt_tsr
sea_depth
shear_exponent
soiling_losses
thrust_coefficient
weibull_k
wind_speed_50m

Setup: Check of root problem complete.
##############################################



{'cycles': [],
 'dangling_params': ['RNA_mass',
  'air_density',
  'altitude',
  'array_losses',
  'availability',
  'blade_number',
  'cut_in_wind_speed',
  'cut_out_wind_speed',
  'max_efficiency',
  'max_power_coefficient',
  'max_tip_speed',
  'multiplier',
  'opt_tsr',
  'sea_depth',
  'shear_exponent',
  'soiling_losses',
  'thrust_coefficient',
  'weibull_k',
  'wind_speed_50m'],
 'driver_issues': {},
 'mode': ('fwd', 'fwd'),
 'mpi': [],
 'no_connect_comps': [],
 'no_unknown_comps': [],
 'out_of_order': [],
 'recorders': [],
 'relevant_pbos': [],
 'solver_issues': None,
 'ubcs': [],
 'unmarked_pbos': []}

The NREL CSM relies on on a large number of wind turbine and plant parameters including overall wind turbine configuration, rotor, nacelle and tower options as well as plant characteristics including the number of turbines, wind resource characteristics and financial parameters.

In [18]:
# set inputs
# simple test of module
# Turbine inputs
prob['rotor_diameter'] = 126.0
prob['blade_number'] = 3
prob['hub_height'] = 90.0    
prob['machine_rating'] = 5000.0

# Rotor force calculations for nacelle inputs
maxTipSpd = 80.0
maxEfficiency = 0.90201
ratedWindSpd = 11.5064
thrustCoeff = 0.50
airDensity = 1.225

ratedHubPower  = prob['machine_rating'] / maxEfficiency 
rotorSpeed     = (maxTipSpd/(0.5*prob['rotor_diameter'])) * (60.0 / (2*np.pi))
prob['rotor_thrust']  = airDensity * thrustCoeff * np.pi * prob['rotor_diameter']**2 * (ratedWindSpd**2) / 8
prob['rotor_torque'] = ratedHubPower/(rotorSpeed*(np.pi/30))*1000

prob['year'] = 2009
prob['month'] = 12

# AEP inputs
prob['max_tip_speed'] = 80.0 #Float(units = 'm/s', iotype='in', desc= 'maximum allowable tip speed for the rotor')
prob['max_power_coefficient'] = 0.488 #Float(iotype='in', desc= 'maximum power coefficient of rotor for operation in region 2')
prob['opt_tsr'] = 7.525 #Float(iotype='in', desc= 'optimum tip speed ratio for operation in region 2')
prob['cut_in_wind_speed'] = 3.0 #Float(units = 'm/s', iotype='in', desc= 'cut in wind speed for the wind turbine')
prob['cut_out_wind_speed'] = 25.0 #Float(units = 'm/s', iotype='in', desc= 'cut out wind speed for the wind turbine')
prob['altitude'] = 0.0 #Float(units = 'm', iotype='in', desc= 'altitude of wind plant')
prob['air_density'] = 0 #Float(units = 'kg / (m * m * m)', iotype='in', desc= 'air density at wind plant site')  # default air density value is 0.0 - forces aero csm to calculate air density in model
prob['max_efficiency'] = 0.902 #Float(iotype='in', desc = 'maximum efficiency of rotor and drivetrain - at rated power')
prob['thrust_coefficient'] = 0.5 #Float(iotype='in', desc='thrust coefficient at rated power')
prob['soiling_losses'] = 0.0
prob['array_losses'] = 0.1
prob['availability'] = 0.941
prob['turbine_number'] = 100
prob['shear_exponent'] = 0.1
prob['wind_speed_50m'] = 8.02
prob['weibull_k']= 2.15

# Finance, BOS and OPEX inputs
prob['RNA_mass'] = 256634.5 
prob['sea_depth'] = 20.0
prob['multiplier'] = 1.0


We can now simulate the overall wind plant cost of energy.

In [19]:
prob.run()

We then print out the resulting cost values:

In [20]:
print("Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines")
for io in root.unknowns:
    print(io + ' ' + str(root.unknowns[io]))

Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines
machine_rating 5000.0
rotor_diameter 126.0
hub_height 90.0
turbine_number 100.0
year 2009.0
month 12.0
bos_costs 764764734.402
bos_breakdown_development_costs 19816413.6622
bos_breakdown_preparation_and_staging_costs 14155377.3927
bos_breakdown_transportation_costs 149852024.568
bos_breakdown_foundation_and_substructure_costs 212330660.89
bos_breakdown_electrical_costs 214524224.858
bos_breakdown_assembly_and_installation_costs 70776886.9634
bos_breakdown_soft_costs 0.0
bos_breakdown_other_costs 83309146.0675
turbine_cost 5383539.544
rotor_cost 1133729.26814
rotor_mass 87595.9193358
turbine_mass 747093.461862
coe 0.120210380123
lcoe 0.112342941751
avg_annual_opex 45280723.0581
opex_breakdown_preventative_opex 34279801.5634
opex_breakdown_corrective_opex 9104838.70968
opex_breakdown_lease_opex 1896082.78501
opex_breakdown_other_opex 0.0
net_aep 1600128912.66
rated_wind_speed 11.5064484118
rated_rotor_speed 12.

###### A land-based project example

Now we can adjust input values to explore the impacts on LCOE.  We began with the NREL 5 MW reference turbine at an offshore site with a sea-depth of 20 m. Let's adjust the inputs to use the same turbine in a land-based project.  To do this, two parameters need to be adjusted: the boolean flag for an offshore project needs to be set to false, and the sea-depth should now be 0 m.

In [21]:
prob['sea_depth'] = 0.0

Now we can re-run the assembly and print the new results.

In [22]:
prob.run()

print("Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines")
for io in root.unknowns:
    print(io + ' ' + str(root.unknowns[io]))

Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines
machine_rating 5000.0
rotor_diameter 126.0
hub_height 90.0
turbine_number 100.0
year 2009.0
month 12.0
bos_costs 308453311.95
bos_breakdown_development_costs 13868377.0651
bos_breakdown_preparation_and_staging_costs 39936721.9216
bos_breakdown_transportation_costs 149852024.568
bos_breakdown_foundation_and_substructure_costs 12077370.8762
bos_breakdown_electrical_costs 76514079.7213
bos_breakdown_assembly_and_installation_costs 16204737.7981
bos_breakdown_soft_costs 0.0
bos_breakdown_other_costs 0.0
turbine_cost 5307582.34094
rotor_cost 1121386.83519
rotor_mass 87595.9193358
turbine_mass 747093.461862
coe 0.0702776033197
lcoe 0.0634748312694
avg_annual_opex 19579732.4312
opex_breakdown_preventative_opex 11997930.5472
opex_breakdown_corrective_opex 5730692.59962
opex_breakdown_lease_opex 1851109.28442
opex_breakdown_other_opex 0.0
net_aep 1600128912.66
rated_wind_speed 11.5064484118
rated_rotor_speed 12.126090

Note that the cost of energy has significantly reduced!  Since we did not change the wind resource or turbine power curve , the AEP is the same while the costs have been reduced.  

The turbine cost is about 5% less since there is a mark-up in the cost model for an offshore turbine.  The balance of station costs and operational expenditures, though, are about half of the offshore costs which is fairly representative of actual projects.  The costs for offshore projects to access the plants, support structures for the turbines, the costs to install them, and other costs are substantially higher than for land-based projects.

###### A more realistic land-based example

Of course, the NREL 5 MW Class I reference turbine is a bulky machine and not likely to be used in a real wind project.  Similarly, using our wind resource is quite high for most land-based applications.  Let's change our turbine specifications to match a turbine meant for land-based applications; say an IEC class III machine and similar resource.  Review the inputs above and adjust those that need to be adjusted for a 2 MW machine at a low-wind speed site. 

(Hint you can cut and paste all the inputs from above and delete those that you aren't adjusting).

In [23]:
# 2 MW turbine specifications and plant attributes
# Turbine inputs
prob['rotor_diameter'] = 120.0
prob['blade_number'] = 3
prob['hub_height'] = 90.0    
prob['machine_rating'] = 2300.0

# Rotor force calculations for nacelle inputs
maxTipSpd = 80.0
maxEfficiency = 0.90201
ratedWindSpd = 11.5064
thrustCoeff = 0.50
airDensity = 1.225

ratedHubPower  = prob['machine_rating'] / maxEfficiency 
rotorSpeed     = (maxTipSpd/(0.5*prob['rotor_diameter'])) * (60.0 / (2*np.pi))
prob['rotor_thrust']  = airDensity * thrustCoeff * np.pi * prob['rotor_diameter']**2 * (ratedWindSpd**2) / 8
prob['rotor_torque'] = ratedHubPower/(rotorSpeed*(np.pi/30))*1000

# AEP inputs
prob['max_tip_speed'] = 80.0 #Float(units = 'm/s', iotype='in', desc= 'maximum allowable tip speed for the rotor')
prob['availability'] = 0.97
prob['turbine_number'] = 100
prob['shear_exponent'] = 0.143
prob['wind_speed_50m'] = 6.5
prob['weibull_k']= 2.1


Now we can re-run the assembly and print the results

In [24]:
prob.run()

print("Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines")
for io in root.unknowns:
    print(io + ' ' + str(root.unknowns[io]))

Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines
machine_rating 2300.0
rotor_diameter 120.0
hub_height 90.0
turbine_number 100.0
year 2009.0
month 12.0
bos_costs 94854288.9329
bos_breakdown_development_costs 5702191.69485
bos_breakdown_preparation_and_staging_costs 17074041.061
bos_breakdown_transportation_costs 13679987.6115
bos_breakdown_foundation_and_substructure_costs 11610853.9685
bos_breakdown_electrical_costs 31484296.7453
bos_breakdown_assembly_and_installation_costs 15302917.8518
bos_breakdown_soft_costs 0.0
bos_breakdown_other_costs 0.0
turbine_cost 3673717.25328
rotor_cost 985514.903446
rotor_mass 78322.9612083
turbine_mass 612421.926102
coe 0.0767400316867
lcoe 0.0687397334778
avg_annual_opex 9786203.23687
opex_breakdown_preventative_opex 6138895.17304
opex_breakdown_corrective_opex 2700164.23712
opex_breakdown_lease_opex 947143.826698
opex_breakdown_other_opex 0.0
net_aep 799306988.873
rated_wind_speed 9.32728001239
rated_rotor_speed 12.732395

Our cost of energy has improved further; note that our turbine (based on the Siemens SWT-2.3-120) has a rotor diameter nearly the same as our offshore 5 MW turbine.  Low-wind speed technology continues to chase larger rotor diameters relative to rated power (or lower specific powers) so that they can have a higher capacity factor which ultimately allows wind projects to be more profitable and economically feasible even at sites with low wind speeds.

What was our capacity factor for the 5 MW turbine project versus our new 2 MW turbine project?  We didn't print out our capacity factor before and we've changed the turbine.  Lucky for us, ipython will let us update this without too much headache! 

(Hint add capacity factor as a printed output in both cases and reexcute the print statement.  Note you will have to reexcute the earlier input settings in the first case or you will be running the lcoe assembly with our new inputs settings).

In [25]:
print("Resulting capacity factor:")
print(root.unknowns['capacity_factor'])

Resulting capacity factor:
39.6717782843


As we can see, the capacity factor for the turbine is much higher for our 2 MW turbine project versus the 5 MW turbine project despite the significantly higher wind resource is the former case.  This is due to the the low-wind speed technology / low specific power / high rotor diameter to power rating ratio.  This simple cost and scaling model (though based on somewhat outdated technology) is great for exploring these types of questions without much computational expense.

In the next tutorial, we will apply more sophisticated analyses to this same model to demonstrate the capabilities of OpenMDAO and DAKOTA for MDAO analysis.